In [1]:
# Points à discuter avec Corentin:
# 1. Méthodes à comparer :
    # a) Méthode de traitement de texte (tf-idf-supervisée et une méthode non supervisée (par ex lda et lda vis)), et
    # b) méthode de prédiction : rf seulement (XGBOOST ne gère pas quand y a n-dimensions)
        # Alternative possible one vs rest (on entraîne un des Tags vs les autres)

# Dans cet exercice : 
    # Approche supervisée = retrouver les tags 
    # Approche non supervisée = prévoir des tags. On ne passe pas par un dataset déjà crée : on crée un tag à partir du texte
        # on peut utiliser : nmf (non negative matrice factorisation) : classement par topics (définis = clusters)
        # lda (latent dirichlet allocation) : assez proche de nmf. 
        # Essayer ldavis qui permet de visualiser les topics générés par lda


# 2. Elaboration base line (mesure performances) : J'ai réussi si je transforme y en table de 1 dimension
    # Est ce ok de procéder comme tel pour la mesure de la perf ? etant entendu que pour la prédiction, je garde
    # y en n-dimensions : ce qui a été choisi numpy.argmin() / numpy.argmax() ==> Sans ce cas pas vraiment le bon choix
    # Sinon, comment on fait ?
# https://scikit-learn.org/stable/modules/model_evaluation.html
# Multilabel
# Accuracy : Compte le score comme OK, si on a prédit correctement l'ensemble des scores : assez conservatif
# On peut se construire sa métrique
# Proportion des tags prédits vs ceux à l'origine

# 3. Output de la prédiction : un peu comme pour le projet de reco de ciné, on va chercher l'indice de la
    # Questionnaire : on saisit le titre et le corps du texte
    # En réponse, on obtient un ou plusieurs Tags Est ce bien cela ?

# Pour suivre les modifications du code final à déployer, 
# utiliser un logiciel de gestion de versions, par exemple Git.
# ==>  A mettre en place maintenant

import warnings
warnings.simplefilter("ignore")
warnings.warn("deprecated", DeprecationWarning)
warnings.simplefilter("ignore")

In [2]:
# Etape 1 : import librairie et données sources
import pandas as pd
import numpy as np
df = pd.read_csv("P6QueryResults4.csv", sep=",")

In [3]:
# Etape 2 : Creation de la matière sur laquelle on va travailler

# Rappel : L'objectif consiste à Prédire des Tags, sur la base :
# Des champs 'Body' et 'Title' qui constituent la variable X
# Du champs 'Tags' qui constitue la variable Y à prédire
df['Text'] = df['Body'].astype(str) + df['Title'].astype(str)
df['Tags'] = df['Tags'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3869 entries, 0 to 3868
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Id      3869 non-null   int64 
 1   Body    3869 non-null   object
 2   Title   3869 non-null   object
 3   Tags    3869 non-null   object
 4   Text    3869 non-null   object
dtypes: int64(1), object(4)
memory usage: 151.3+ KB


In [4]:
# Etape 3 : Import librairie pour nettoyage texte et tf-idf
import re
import nltk
import pickle
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

In [5]:
# Définition des X et y
X, y = df.Text, df.Tags

In [6]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;?*]')
BAD_SYMBOLS_RE1 = re.compile('[^0-9a-z #+_-]')
STOPWORDS = list((stopwords.words('english')))

def text_prepare_WithSoup(text, join_symbol):
    """
        text: a string

        return: modified initial string
    """
    # 1. Remove HTML
    text = BeautifulSoup(text,features="lxml").get_text()
    
    # lowercase text
    text = text.lower()

    # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = re.sub(REPLACE_BY_SPACE_RE, " ", text,)

    # delete symbols which are in BAD_SYMBOLS_RE from text
    text = re.sub(BAD_SYMBOLS_RE1, " ", text)
    text = re.sub(r'\s+', " ", text)

    # delete stopwords from text
    text = f'{join_symbol}'.join(
        [i for i in text.split() if i not in STOPWORDS])

    return text


tests = ["SQL Server - any equivalent of Excel's CHOOSE function VB.NET. ?",
         "How to free c++ memory vector<int> * arr?"]
for test in tests:
    print(text_prepare_WithSoup(test, ' '))

sql server - equivalent excel choose function vb net
free c++ memory vector arr


In [7]:
BAD_SYMBOLS_RE2 = re.compile('[^0-9a-z .#+_-]') # A la différence de 'BAD_SYMBOLS_RE2', ici, on garde le '.'
# 'VB.NET' devient 'vb.net'
def text_prepare_NoSoup(text, join_symbol):
    """
        text: a string

        return: modified initial string
    """
    # lowercase text
    text = text.lower()

    # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = re.sub(REPLACE_BY_SPACE_RE, " ", text,)

    # delete symbols which are in BAD_SYMBOLS_RE from text
    text = re.sub(BAD_SYMBOLS_RE2, " ", text)
    text = re.sub(r'\s+', " ", text)

    # delete stopwords from text
    text = f'{join_symbol}'.join(
        [i for i in text.split() if i not in STOPWORDS])

    return text
for test in tests:
    print(text_prepare_NoSoup(test, ' '))

sql server - equivalent excel choose function vb.net.
free c++ memory vector int arr


In [8]:
X_cleaned = [text_prepare_WithSoup(X, ' ') for X in X]
y_cleaned = [text_prepare_NoSoup(y, ' ') for y in y]

In [9]:
X[0]

'<p>I want to use a <code>Track-Bar</code> to change a <code>Form</code>\'s opacity.</p>\n<p>This is my code:</p>\n<pre class="lang-cs prettyprint-override"><code>decimal trans = trackBar1.Value / 5000;\nthis.Opacity = trans;\n</code></pre>\n<p>When I build the application, it gives the following error:</p>\n<blockquote>\n<p>Cannot implicitly convert type <code>decimal</code> to <code>double</code></p>\n</blockquote>\n<p>I have tried using <code>trans</code> and <code>double</code> but then the <code>Control</code> doesn\'t work. This code worked fine in a past VB.NET project.</p>\nConvert Decimal to Double'

In [10]:
X_cleaned[0]

'want use track-bar change form opacity code decimal trans trackbar1 value 5000 opacity trans build application gives following error cannot implicitly convert type decimal double tried using trans double control work code worked fine past vb net project convert decimal double'

In [11]:
y[50:65]

50                                  <c++><berkeley-db>
51                                <svn><collaboration>
52                     <c#><.net><sorting><dictionary>
53            <sql><database><oracle><version-control>
54                                     <security><php>
55                     <c++><oop><class><nested-class>
56                          <language-agnostic><types>
57                                       <python><xml>
58         <string><language-agnostic><cross-platform>
59                                 <email><email-spam>
60    <java><generics><warnings><casting><type-safety>
61                                    <search><lucene>
62                       <ios><objective-c><landscape>
63                                          <com><vb6>
64       <internet-explorer><windows-mobile><pocketpc>
Name: Tags, dtype: object

In [12]:
y_cleaned [50:65]

['c++ berkeley-db',
 'svn collaboration',
 'c# .net sorting dictionary',
 'sql database oracle version-control',
 'security php',
 'c++ oop class nested-class',
 'language-agnostic types',
 'python xml',
 'string language-agnostic cross-platform',
 'email email-spam',
 'java generics warnings casting type-safety',
 'search lucene',
 'ios objective-c landscape',
 'com vb6',
 'internet-explorer windows-mobile pocketpc']

In [13]:
# Etape 5 : Conversion des données textuelles et valeur numériques
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
vectorizer_x = CountVectorizer(analyzer="word",
                             max_features=1500,
                             # min_df=0.05,
                             max_df=0.9,
                             stop_words=stopwords.words('english'),
                             tokenizer = None,
                             preprocessor = None,
                             )
            
X = vectorizer_x.fit_transform(X_cleaned).toarray()

In [15]:
# On obtient les noms colonnes variable X
ColName_X = list(vectorizer_x.vocabulary_.keys())[:1500]

In [16]:
vectorizer_y = CountVectorizer(analyzer="word",
                             max_features=50,
                             # min_df=0.05,
                             max_df=0.85,
                             stop_words=stopwords.words('english'),
                             tokenizer = None,
                             preprocessor = None,
                             )

y=vectorizer_y.fit_transform(y_cleaned).toarray()

In [17]:
# On obtient les noms colonnes variable y
ColName_y = list(vectorizer_y.vocabulary_.keys())[:50]

In [18]:
tfidf_vocab_y = vectorizer_y.get_feature_names()
print(tfidf_vocab_y)

['2005', '2008', 'agnostic', 'algorithm', 'apache', 'asp', 'build', 'control', 'css', 'data', 'database', 'design', 'file', 'html', 'interface', 'java', 'javascript', 'jquery', 'language', 'linq', 'linux', 'macos', 'multithreading', 'mvc', 'mysql', 'net', 'oop', 'performance', 'php', 'python', 'regex', 'ruby', 'security', 'server', 'services', 'sql', 'string', 'studio', 'svn', 'testing', 'unit', 'user', 'vb', 'version', 'visual', 'web', 'windows', 'winforms', 'wpf', 'xml']


In [19]:
X.shape

(3869, 1500)

In [20]:
X[0]

array([0, 0, 0, ..., 0, 0, 0])

In [21]:
y.shape

(3869, 50)

In [22]:
y[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [23]:
# Etape 6 : Process TF-IDF
# 6.1 Pour X
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()


In [24]:
X.shape

(3869, 1500)

In [25]:
X[0:1]

array([[0., 0., 0., ..., 0., 0., 0.]])

In [26]:
y.shape

(3869, 50)

In [27]:
y[0:2]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0]])

In [28]:
# Etape 7 : Mise en place des ensemble train test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [29]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2708, 1500)
(1161, 1500)
(2708, 50)
(1161, 50)


In [30]:
# Etape 8 : Entrainement classification des textes et prédiction des tags
# Préalablement, on fait un GridSearchCV pour recherche paramètres à optimiser
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

param_gridRF = {"max_depth": [3, 5],  # Maximum number of levels in tree
                "min_samples_split": [3, 10],# Minimum number of samples required to split a node
                }

rfc = RandomForestClassifier(random_state=42, oob_score=True)

In [31]:
# GridSearchCV : La fonction RF fonctionne, mais pas les fonctions de calcul de perf.
# modelrfc = GridSearchCV(rfc,
                       # param_gridRF,
                       # n_jobs=-1,
                       # cv=5
                       # )
# modelrf.fit(X_train, y_train)

In [32]:
# print(modelrfc.best_params_)
# Pas de raison que le grid search ne fonctionne pas pour un y n-dimensions

In [33]:
# Prédiction sur la base des meilleurs paramètres
classifier = RandomForestClassifier(max_depth = 5, min_samples_split = 10)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=10,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [34]:
# Prédiction modèle y = 1 dimension pour obtenir les performances
y_pred = classifier.predict(X_test)
y_pred.shape

(1161, 50)

In [35]:
from sklearn.metrics import f1_score

In [36]:
# On va se créer notre propre fonction de calcul de perf
# Est ce que les deux y_pred et y_tests sont identiques ?
comparaison = y_pred == y_test
equal_arrays = comparaison.all()
print(equal_arrays)
# La réponse est non, ce qui est une bonne nouvelle !

False


In [37]:
# Moyenne des différences
y_dif_mean = np.mean(y_test - y_pred)
# On a une différence moyenne de 3%, ce qui implique que la prédiction est assez juste
print(y_dif_mean)
print(y_dif_mean.shape)

0.0321619293712317
()


In [38]:
y_diff = np.subtract(y_test,y_pred)
print(y_diff.shape)

(1161, 50)


In [39]:
# STd dev des difference
np.std(y_test - y_pred)
# On a une erreur moyenne de 20 %, ce qui implique une variation très grande des types d'erreur

0.19222410468812415

In [40]:
# Essayer d'établir une matrice de confusion ou une sorte d'équivalent

In [41]:
# print(f1_score(y_test, y_pred)) # Devrait accepter y n-dimensions

In [42]:
# Etape 9 : Evaluation du modèle : NE FONCTIONNE PAS
# from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
# print(confusion_matrix(y_test,y_pred))
# print(classification_report(y_test,y_pred))
# print(accuracy_score(y_test, y_pred)) # Devrait accepter y n-dimensions

In [43]:
# *** ICI *** Effectuer methode BERT et comparer avec TF-IDF

In [44]:
# Etape 10 : Méthode non supervisée, création de clusters de tags par méthode non supervisées

In [53]:
# 10.1 Clusters via K-mean
from sklearn.cluster import KMeans
# Perform k-means clustering
num_clusters = 30
kmeans = KMeans(n_clusters=num_clusters)
# Use k-keans to make sentiment label clustering
kmeans.fit(y)
# get cluster assignments; a label (0 or 1) for each review
labels = kmeans.labels_.tolist()

In [54]:
# Original 
sorted_index_centroids = kmeans.cluster_centers_.argsort()[:,::-1]

# ***ICI
# Pourquoi est ce vide ?

In [55]:
# for i in range(num_clusters):
for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    # replace 10 with n words per cluster
    for ind in sorted_index_centroids[i, :5]:
        print(' %s' % tfidf_vocab_y[ind].split(' ')[0], end=',')
    print()  # add whitespace
    print()  # add whitespace

# 2020 08 31 - Cluster à améliorer - source = clean_train_reviews

Cluster 0 words: testing, unit, net, language, agnostic,

Cluster 1 words: algorithm, macos, apache, xml, web,

Cluster 2 words: net, winforms, windows, wpf, services,

Cluster 3 words: sql, server, database, net, services,

Cluster 4 words: net, visual, studio, asp, vb,

Cluster 5 words: java, xml, string, windows, regex,

Cluster 6 words: php, mysql, oop, performance, apache,

Cluster 7 words: python, linux, xml, windows, macos,

Cluster 8 words: asp, net, javascript, web, services,

Cluster 9 words: ruby, windows, string, mysql, design,

Cluster 10 words: database, sql, mysql, design, performance,

Cluster 11 words: sql, server, 2005, database, 2008,

Cluster 12 words: language, agnostic, algorithm, regex, oop,

Cluster 13 words: linq, sql, net, asp, database,

Cluster 14 words: windows, file, svn, linux, server,

Cluster 15 words: net, asp, mvc, testing, html,

Cluster 16 words: svn, version, control, visual, database,

Cluster 17 words: data, algorithm, database, wpf, xml,

Cluste

In [ ]:
# 10.2 Cluster via LDA

In [48]:
En attente

SyntaxError: invalid syntax (<ipython-input-48-0f1752d7e310>, line 1)

In [ ]:
# Maintenant que l'on a les paramètres optimaux et une baseline, on va exécuter le modèle
# mais avec y en n-dimensions

In [ ]:
# Prédiction sur la base des meilleurs paramètres
modelrf = RandomForestClassifier(max_depth=5, min_samples_split=10)
modelrf.fit(X_train, y_train)

In [ ]:
# Prédiction modèle y = 1 dimension pour obtenir les performances
y_pred_rf = modelrf.predict(X_test)

In [ ]:
# Etape 10 : Sauvegarde du modèle devra être adapté si BERT = meilleur
with open('P6text_classifier', 'wb') as picklefile:
    pickle.dump(modelrf,picklefile)

In [ ]:
# Etape 11 : Chargement du modèle pour prédire les tags et évaluer sa performance
with open('P6text_classifier', 'rb') as training_model:
    model = pickle.load(training_model)